In [ ]:
"""
Created on Thu Feb 08 10:42 2024

Look at some 2D variables in the Casimir runs

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

import gsw

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'
mask_path = '/data/cburgard/TOOLS/'

In [5]:
ocean_masks = xr.open_dataset(mask_path + 'basin_masks_orca1_nemo4p2.nc')

In [6]:
file_open_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
file_closed_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')

file_open_V_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_V.nc').mean('time_counter')
file_closed_V_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_V.nc').mean('time_counter')

In [7]:
file_open_T = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc')
file_closed_T = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc')

file_open_V = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_V.nc')
file_closed_V = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_V.nc')

In [8]:
file_open_mean10 = file_open_mean10.where(file_open_mean10['so'] > 0)
file_open_V_mean10 = file_open_V_mean10#.where(file_open_mean10['so'].rename({'deptht':'depthv'}) > 0)

file_open_T = file_open_T.where(file_open_T['so'] > 0)

In [9]:
file_open_ice = xr.open_mfdataset(inputpath_open+'/n42openc_00910101_01001231_1M_icemod.nc')
file_closed_ice = xr.open_mfdataset(inputpath_closed+'/n42tm21_00910101_01001231_1M_icemod.nc')

In [10]:
file_open_ice_02 = file_open_ice.where(file_open_ice['time_counter.month'] == 2, drop=True).mean('time_counter').load()
file_open_ice_09 = file_open_ice.where(file_open_ice['time_counter.month'] == 9, drop=True).mean('time_counter').load()

file_closed_ice_02 = file_closed_ice.where(file_closed_ice['time_counter.month'] == 2, drop=True).mean('time_counter').load()
file_closed_ice_09 = file_closed_ice.where(file_closed_ice['time_counter.month'] == 9, drop=True).mean('time_counter').load()

In [11]:
lon = file_open_mean10.nav_lon
lat = file_open_mean10.nav_lat

PLOTS

In [ ]:
T_30W_closed = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')
T_30W_open = file_open_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')

In [12]:
# checking if the line is right
#ref = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).isel(deptht=0)
ref = ocean_masks['pacific']
plt.figure()

llon = lon
llat = lat

proj = ccrs.PlateCarree(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [13]:
def compute_density(da):
    depth = da['e3t'].cumsum('deptht')
    S = da['so']
    T = da['thetao']
    return gsw.rho(S,T,depth)

In [14]:
def compute_sigma2(da):
    S = da['so']
    T = da['thetao']
    return gsw.density.sigma2(S, T)

In [15]:
def transect_along_ocean(var, lon, lon_min, lon_max, lat, lat_max, cpalette, varname):

    if lat_max == 90:
        plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max)).mean('x')
        plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max)).mean('x')
    else:
        plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max), drop=True).mean('x')
        plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max), drop=True).mean('x')
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*1.5))

    abs1 = ax[0].contourf(plot_var_closed.y,-1*plot_var_closed.deptht, plot_var_closed, cmap=cpalette)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title(varname)
    
    abs2 = ax[1].contourf(plot_var_open.y,-1*plot_var_open.deptht, plot_var_open, cmap=cpalette)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].contourf(plot_var_open.y,-1*plot_var_open.deptht, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

    plt.tight_layout()
    return f

In [16]:
def streamfunction_along_ocean(vo_open, vo_closed, ocean, cpalette):

    if ocean == 'indo-pacific':
        vmass_sum_open = vo_open.where(np.isfinite(ocean_masks['indian']) | np.isfinite(ocean_masks['pacific'])).sum('x') / 10**6
        vmass_sum_closed = vo_closed.where(np.isfinite(ocean_masks['indian']) | np.isfinite(ocean_masks['pacific'])).sum('x') / 10**6
    else:
        vmass_sum_open = vo_open.where(np.isfinite(ocean_masks[ocean])).sum('x') / 10**6
        vmass_sum_closed = vo_closed.where(np.isfinite(ocean_masks[ocean])).sum('x') / 10**6

    vmass_sum_open = vmass_sum_open.sel(depthv=vmass_sum_open.depthv[::-1])
    vmass_sum_closed = vmass_sum_closed.sel(depthv=vmass_sum_closed.depthv[::-1])

    plot_var_open = -1*vmass_sum_open.cumsum('depthv')
    plot_var_closed = -1*vmass_sum_closed.cumsum('depthv')
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*1.5))

    ax_limit0 = 20 #np.quantile(np.array([np.abs(plot_var_open),np.abs(plot_var_closed)]), 0.99)
    
    abs1 = ax[0].pcolormesh(plot_var_closed.y,-1*plot_var_closed.depthv, plot_var_closed, cmap=cpalette, vmin=-ax_limit0, vmax=ax_limit0)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title('Streamfunction '+ocean)

    abs2 = ax[1].pcolormesh(plot_var_open.y,-1*plot_var_open.depthv, plot_var_open, cmap=cpalette, vmin=-ax_limit0, vmax=ax_limit0)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].pcolormesh(plot_var_open.y,-1*plot_var_open.depthv, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

    plt.tight_layout()
    return f

In [17]:
def maps_compare3(lon,lat,ref,modif,name,cmap,region,legend,lat_lim=-50):
    
    f = plt.figure(figsize=(8.25*1.5,8.25/1.5))
    #f = plt.figure()
    #f.suptitle(str(time_in.values)[0:16],fontsize=22)

    if region == 'global':
        proj = ccrs.Mollweide(central_longitude=0)
        wrap_ref = ref
        wrap_lon = lon
        wrap_modif = modif
    elif region == 'Ant':
        proj = ccrs.SouthPolarStereo(central_longitude=0)
        wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)
        wrap_modif, wrap_lon = modif, lon #add_cyclic_point(modif.values,coord=lon,axis=1)

        theta = np.linspace(0, 2*np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpl.path.Path(verts * radius + center)        

    #### REFERENCE
    ax1 = plt.subplot(1, 3, 1, projection=proj)
    abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),cmap=cmap,rasterized=True)
    ax1.coastlines(resolution='110m', linewidth=0.5)
    
    if region == 'global':
        ax1.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    elif region == 'Ant':
        ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
        ax1.set_boundary(circle, transform=ax1.transAxes)

    if legend=='yes':
        cbar = f.colorbar(abso0, ax=ax1, shrink=0.3,orientation='vertical',extend='both')
        cbar.set_label(name,rotation=90)  

    #### MODIFICATION
    ax2 = plt.subplot(1, 3, 2, projection=proj)
    ax2.coastlines(resolution='110m', linewidth=0.5)
    abso = ax2.pcolormesh(wrap_lon,lat,wrap_modif,transform=ccrs.PlateCarree(),cmap=cmap,rasterized=True)

    if region == 'global':
        ax2.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    elif region == 'Ant':
        ax2.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
        ax2.set_boundary(circle, transform=ax2.transAxes)

    if legend=='yes':
        cbar = f.colorbar(abso0, ax=ax2, shrink=0.3,orientation='vertical',extend='both')
        cbar.set_label(name,rotation=90)    

    #### DIFFERENCE
    abs_lim = abs(modif - ref).quantile(0.99)
    ax3 = plt.subplot(1, 3, 3, projection=proj)
    ax3.coastlines(resolution='110m', linewidth=0.5)
    diff1 = ax3.pcolormesh(wrap_lon,lat,wrap_modif - wrap_ref,transform=ccrs.PlateCarree(),cmap=mpl.cm.coolwarm,rasterized=True,vmin=-abs_lim,vmax=abs_lim)
    
    if region == 'global':
        ax3.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    elif region == 'Ant':
        ax3.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
        ax3.set_boundary(circle, transform=ax3.transAxes)

    if legend=='yes':
        cbar = f.colorbar(diff1, ax=ax3, shrink=0.3,orientation='vertical',extend='both')
        cbar.set_label('Modif - ref '+name,rotation=90)    


    
    f.tight_layout()
    
    return f
    

In [ ]:
### ATLANTIC

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -31., -29., lat, 90, cmocean.cm.thermal,'Conservative Temperature [°C]')
f_temp.savefig(plot_path + 'transect_temp_Atl.pdf')

In [ ]:
f_sal = transect_along_ocean('so', lon, -31., -29., lat, 90,cmocean.cm.haline,'Absolute Salinity [g/kg]')
f_sal.savefig(plot_path + 'transect_sal_Atl.pdf')

In [ ]:
file_closed_mean10['rho'] = compute_density(file_closed_mean10).load()
file_open_mean10['rho'] = compute_density(file_open_mean10).load()

In [ ]:
f_rho = transect_along_ocean('rho', lon, -31., -29., lat, 90, cmocean.cm.dense,'In-situ density [kg/m3]')

In [18]:
file_closed_mean10['sigma2'] = compute_sigma2(file_closed_mean10).load()
file_open_mean10['sigma2'] = compute_sigma2(file_open_mean10).load()

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -31., -29., lat, 90,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')
f_sigma2.savefig(plot_path + 'transect_sigma2_Atl.pdf')

In [ ]:
### PACIFIC

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -171, -169.,lat, 90,cmocean.cm.thermal,'Conservative Temperature [°C]')
f_temp.savefig(plot_path + 'transect_temp_Pac.pdf')

In [ ]:
f_sal = transect_along_ocean('so', lon,  -171, -169.,lat, 90,cmocean.cm.haline,'Absolute Salinity [g/kg]')
f_sal.savefig(plot_path + 'transect_sal_Pac.pdf')

In [ ]:
f_rho = transect_along_ocean('rho', lon, -171, -169.,lat, 90,cmocean.cm.dense,'In-situ density [kg/m3]')

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -171, -169.,lat, 90,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')
f_sigma2.savefig(plot_path + 'transect_sigma2_Pac.pdf')

In [ ]:
### WEDDELL SEA

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -51, -49,lat, -50,cmocean.cm.thermal,'Conservative Temperature [°C] Weddell Sea')
f_temp.savefig(plot_path + 'transect_temp_Wed.pdf')

In [ ]:
f_sal = transect_along_ocean('so', lon,  -51, -49,lat,  -50,cmocean.cm.haline,'Absolute Salinity [g/kg] Weddell Sea')
f_sal.savefig(plot_path + 'transect_sal_Wed.pdf')

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -51, -49,lat, -50,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3] Weddell Sea')
f_sigma2.savefig(plot_path + 'transect_sigma2_Wed.pdf')

In [ ]:
### ROSS SEA

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -180, -179,lat, -50,cmocean.cm.thermal,'Conservative Temperature [°C] Ross Sea')
f_temp.savefig(plot_path + 'transect_temp_Ross.pdf')

In [ ]:
f_sal = transect_along_ocean('so', lon,  -180, -179,lat,  -50,cmocean.cm.haline,'Absolute Salinity [g/kg] Ross Sea')
f_sal.savefig(plot_path + 'transect_sal_Ross.pdf')

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -180, -179,lat, -50,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3] Ross Sea')
f_sigma2.savefig(plot_path + 'transect_sigma2_Ross.pdf')

In [ ]:
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)

In [ ]:
### STREAM FUNCTION

In [ ]:
f_stream_indopac = streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'indo-pacific',mpl.cm.coolwarm)
f_stream_indopac.savefig(plot_path + 'streamfunction_indopac.png')

In [ ]:
f_stream_atl = streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'atlantic',mpl.cm.coolwarm)
f_stream_atl.savefig(plot_path + 'streamfunction_atl.png')

In [ ]:
f_stream_glob = streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'global',mpl.cm.coolwarm)
f_stream_glob.savefig(plot_path + 'streamfunction_glob.png')

In [ ]:
plt.close('all')

In [ ]:
### MAPS

In [19]:
f_sic_09 = figf.maps_compare3(lon,lat,file_closed_ice_09['siconc'],file_open_ice_09['siconc'],'September sea-ice concentration',cmocean.cm.ice,'Ant','yes',lat_lim=-50)
f_sic_09.savefig(plot_path + 'sic_09_Ant.pdf')

In [20]:
f_sic_02 = figf.maps_compare3(lon,lat,file_closed_ice_02['siconc'],file_open_ice_02['siconc'],'February sea-ice concentration',cmocean.cm.ice,'Ant','yes',lat_lim=-50)
f_sic_02.savefig(plot_path + 'sic_02_Ant.pdf')

In [21]:
f_sit_09 = figf.maps_compare3(lon,lat,file_closed_ice_09['sivolu'],file_open_ice_09['sivolu'],'September sea-ice volume',cmocean.cm.ice,'Ant','yes',lat_lim=-50)
f_sit_09.savefig(plot_path + 'sit_09_Ant.pdf')

In [22]:
f_sit_02 = figf.maps_compare3(lon,lat,file_closed_ice_02['sivolu'],file_open_ice_02['sivolu'],'February sea-ice volume',cmocean.cm.ice,'Ant','yes',lat_lim=-50)
f_sit_02.savefig(plot_path + 'sit_02_Ant.pdf')

In [23]:
fig_mld = figf.maps_compare3(lon,lat,file_closed_mean10['mldr10_1'].load(),file_open_mean10['mldr10_1'].mean('deptht').load(),'Mixed layer depth',cmocean.cm.deep,'Ant','yes',lat_lim=-50)
fig_mld.savefig(plot_path + 'MLD_Ant.pdf')

In [24]:
mask_cells = (file_closed_mean10['so'] > 0).load()
bottom_open = mask_cells.sum('deptht') - 1

mask_cells = (file_open_mean10['so'] > 0).load()
bottom_closed = mask_cells.sum('deptht') - 1

In [28]:
fig_T_bot = figf.maps_compare3(lon,lat,file_closed_mean10['thetao'].isel(deptht=bottom_closed).load(),file_open_mean10['thetao'].isel(deptht=bottom_open).load(),'Conservative bottom temperature [°C]',cmocean.cm.thermal,'Ant','yes',lat_lim=-50)
fig_T_bot.savefig(plot_path+'T_bottom_Ant.png')

In [29]:
fig_S_bot = figf.maps_compare3(lon,lat,file_closed_mean10['so'].isel(deptht=bottom_closed).load(),file_open_mean10['so'].isel(deptht=bottom_open).load(),'Absolute salinity [g/kg]',cmocean.cm.haline,'Ant','yes',lat_lim=-50)
fig_S_bot.savefig(plot_path+'S_bottom_Ant.png')

In [30]:
fig_sigma2_bot = figf.maps_compare3(lon,lat,file_closed_mean10['sigma2'].isel(deptht=bottom_closed),file_open_mean10['sigma2'].isel(deptht=bottom_open),'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]',cmocean.cm.dense,'Ant','yes',lat_lim=-50)
fig_sigma2_bot.savefig(plot_path+'sigma2_bottom_Ant.png')

In [ ]:
#### BINNED STREAM FUNCTION

In [ ]:
### Compute potential density anomaly with reference pressure of 2000 dbar

In [ ]:
file_closed_T['sigma2'] = compute_sigma2(file_closed_T)
file_open_T['sigma2'] = compute_sigma2(file_open_T)

In [ ]:
print(file_closed_T['sigma2'].max().load()) #37.8069389
print(file_closed_T['sigma2'].min().load()) #10.16092161

In [ ]:
print(file_open_T['sigma2'].max().load()) #37.80702688
print(file_open_T['sigma2'].min().load()) #9.813129

In [ ]:
### Interpolate from V to T grid

In [ ]:
sigma2_V_closed = (file_closed_T['sigma2'] + file_closed_T['sigma2'].shift(y=-1)) / 2
sigma2_V_closed.loc[{'y': sigma2_V_closed.y.max()}] = file_closed_T['sigma2'].sel(y=sigma2_V_closed.y.max())

sigma2_V_open = (file_open_T['sigma2'] + file_open_T['sigma2'].shift(y=-1)) / 2
sigma2_V_open.loc[{'y': sigma2_V_open.y.max()}] = file_open_T['sigma2'].sel(y=sigma2_V_open.y.max())

In [ ]:
vtra_closed = file_closed_V['vocetr_eff']
sigma2_V_closed = sigma2_V_closed.rename({'deptht': 'depthv'}).where(np.isfinite(vtra_closed))

vtra_open = file_open_V['vocetr_eff']
sigma2_V_open = sigma2_V_open.rename({'deptht': 'depthv'}).where(np.isfinite(vtra_open))

In [ ]:
print(sigma2_V_closed.max().load()) #37.80693871
print(sigma2_V_closed.min().load()) #10.23064935

In [ ]:
print(sigma2_V_open.max().load()) #37.80702652
print(sigma2_V_open.min().load()) #9.82776096

In [ ]:
### Compute cell volume, depth, bathy

In [ ]:
volu_closed = file_closed_V.e3v * file_closed_V.area
volu_open = file_open_V.e3v * file_open_V.area


In [ ]:
print((volu_closed.sum()/10).load()) #1.363346e+18
print((volu_open.sum()/10).load()) #2.6498255e+18

In [ ]:
e3v_closed = file_closed_V.e3v.where(np.isfinite(vtra_closed))
e3v_open = file_open_V.e3v.where(np.isfinite(vtra_open))

In [ ]:
e3v_closed

In [ ]:
depth_closed = xr.concat([xr.DataArray(data=np.zeros((len(e3v_closed.time_counter),len(e3v_closed.y),len(e3v_closed.x))), dims=['time_counter','y','x']).assign_coords(
    {'time_counter': e3v_closed.time_counter, 'y': e3v_closed.y, 'x': e3v_closed.x, 'depthv': 0}),
    e3v_closed.cumsum('depthv')], dim='depthv')
depth_closed = depth_closed.where(np.isfinite(e3v_closed))

depth_open = xr.concat([xr.DataArray(data=np.zeros((len(e3v_open.time_counter),len(e3v_open.y),len(e3v_open.x))), dims=['time_counter','y','x']).assign_coords(
    {'time_counter': e3v_open.time_counter, 'y': e3v_open.y, 'x': e3v_open.x, 'depthv': 0}),
    e3v_open.cumsum('depthv')], dim='depthv')
depth_open = depth_open.where(np.isfinite(e3v_open))

In [ ]:
depth_mid_closed = (depth_closed + depth_closed .shift(depthv=-1)) / 2
depth_mid_open = (depth_open + depth_open .shift(depthv=-1)) / 2

In [ ]:
bathy_closed = depth_closed.isel(time_counter=9).max('depthv')
bathy_open = depth_open.isel(time_counter=9).max('depthv')

In [ ]:
### double check on the open stuff => needs a mask I guess!

bathy_open.plot()

In [ ]:
bathy = np.isfinite(depth_closed)

In [ ]:
depth_open.isel(time_counter=0,y=100,x=100).load()

In [ ]:
print((volu_closed.sum()/10).load()) #1.363346e+18
print((volu_open.sum()/10).load()) #2.6498255e+18

In [ ]:
file_open_V_mean10['vocetr_eff'].sum('x').plot()

In [ ]:
v_atlsum_open = file_open_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['global'])).sum('x') / 10**6
v_atlsum_closed = file_closed_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['global'])).sum('x') / 10**6

In [ ]:
v_atlsum_open_rev = v_atlsum_open.sel(depthv=v_atlsum_open.depthv[::-1])
v_atlsum_closed_rev = v_atlsum_closed.sel(depthv=v_atlsum_closed.depthv[::-1])

In [ ]:
v_atlsum_closed_rev.plot()

In [ ]:
streamf_atl_open = v_atlsum_open.sel(depthv=v_atlsum_open.depthv[::-1]).cumsum('depthv')
streamf_atl_closed = v_atlsum_closed.sel(depthv=v_atlsum_closed.depthv[::-1]).cumsum('depthv')

In [ ]:
streamf_atl_closed.plot()

In [ ]:
streamf_atl_closed.plot(vmax=20)

In [ ]:
streamf_atl_open_rev = v_atlsum_open_rev.cumsum('depthv')
streamf_atl_closed_rev = v_atlsum_closed_rev.cumsum('depthv').assign_coords({'depthv': -1*v_atlsum_closed_rev.depthv})


In [ ]:
plt.figure()
streamf_atl_closed_rev.plot(vmax=20)

In [ ]:
streamf_atl_open.plot()

In [ ]:
file_open_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['atlantic'])).isel(depthv=0).plot()

In [ ]:
v_atlsum_open.plot()

In [ ]:
v_atlsum_open.cumsum('depthv').plot()

In [ ]:
plt.figure()
v_atlsum_open_rev.cumsum('depthv').plot()